In [1]:
%reset -f
%load_ext autoreload
%autoreload 2

In [45]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from sentinelhub import CRS, BBox, DataCollection, SHConfig
from sentinelhub import SentinelHubCatalog
import os
from dotenv import load_dotenv
import getpass
from pathlib import Path

In [3]:
USERNAME = getpass.getuser()

# Load environment variables from .env file
env_path = Path(f"/data/users/Private/{USERNAME}/configs/shub.env")
if env_path.exists():
    load_dotenv(env_path)
else:
    raise FileNotFoundError(f"The environment file {env_path} does not exist.")

# Your client credentials
sh_client_id = os.getenv("SH_CLIENT_ID")
sh_client_secret = os.getenv("SH_CLIENT_SECRET")

In [4]:
# Create a session
client = BackendApplicationClient(client_id=sh_client_id)
oauth = OAuth2Session(client=client)

In [5]:
# Get token for the session
token = oauth.fetch_token(
    token_url="https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token",
    client_secret=sh_client_secret,
    include_client_id=True,
)

In [6]:
# All requests using this session will have an access token automatically added
resp = oauth.get(
    "https://services.sentinel-hub.com/configuration/v1/wms/instances"
)
print(resp.content)

b'[{"@id":"https://services.sentinel-hub.com/configuration/v1/wms/instances/200b04af-a4d2-4322-958c-e7fea1350e15","id":"200b04af-a4d2-4322-958c-e7fea1350e15","name":"Sentinel 2 WMS template","domainAccountId":"781dd599-0e3f-4269-a4de-8a7dfe3aba76","additionalData":{"showWarnings":true,"showLogo":true,"imageQuality":90},"created":"2024-10-07T07:47:22.745953Z","lastUpdated":"2024-10-07T07:47:22.745953Z","layers":{"@id":"https://services.sentinel-hub.com/configuration/v1/wms/instances/200b04af-a4d2-4322-958c-e7fea1350e15/layers"}},{"@id":"https://services.sentinel-hub.com/configuration/v1/wms/instances/8d17be5a-c9a7-4a5c-9bea-3764d8dd54d5","id":"8d17be5a-c9a7-4a5c-9bea-3764d8dd54d5","name":"Planet Sandbox Data template","domainAccountId":"781dd599-0e3f-4269-a4de-8a7dfe3aba76","additionalData":{"showWarnings":true,"showLogo":true,"imageQuality":90,"disabled":false},"created":"2024-10-07T07:47:22.973496Z","lastUpdated":"2024-10-07T07:47:22.973496Z","layers":{"@id":"https://services.sentinel

In [7]:
config = SHConfig(
    sh_client_id=sh_client_id,
    sh_client_secret=sh_client_secret,
)
print(f"Config made: {config}")

Config made: {
  "instance_id": "",
  "sh_client_id": "********************************f99a",
  "sh_client_secret": "****************************dNn4",
  "sh_base_url": "https://services.sentinel-hub.com",
  "sh_auth_base_url": null,
  "sh_token_url": "https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token",
  "geopedia_wms_url": "https://service.geopedia.world",
  "geopedia_rest_url": "https://www.geopedia.world/rest",
  "aws_access_key_id": "",
  "aws_secret_access_key": "",
  "aws_session_token": "",
  "aws_metadata_url": "https://roda.sentinel-hub.com",
  "aws_s3_l1c_bucket": "sentinel-s2-l1c",
  "aws_s3_l2a_bucket": "sentinel-s2-l2a",
  "opensearch_url": "http://opensearch.sentinel-hub.com/resto/api/collections/Sentinel2",
  "max_wfs_records_per_query": 100,
  "max_opensearch_records_per_query": 500,
  "max_download_attempts": 4,
  "download_sleep_time": 5.0,
  "download_timeout_seconds": 120.0,
  "number_of_download_processes": 1,
  "max_retries": null
}

In [8]:
# Get catalog info
catalog = SentinelHubCatalog(config=config)
catalog.get_info()
print(f"Catalog: {catalog}")

Catalog: <sentinelhub.api.catalog.SentinelHubCatalog object at 0x7f6f87e21690>


In [9]:
# Get collections 
collections = catalog.get_collections()
collections = [
    collection
    for collection in collections
    if not collection["id"].startswith(("byoc", "batch"))
]
s2_coll = collections[2]
print(f"Sentinel 2 L2A collections: {s2_coll}")

Sentinel 2 L2A collections: {'stac_version': '1.0.0', 'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json', 'https://stac-extensions.github.io/eo/v1.0.0/schema.json'], 'type': 'Collection', 'id': 'sentinel-2-l2a', 'title': 'Sentinel 2 L2A', 'description': 'Sentinel 2 imagery processed to level 2A', 'sci:citation': 'Modified Copernicus Sentinel data [Year]/Sentinel Hub', 'license': 'proprietary', 'providers': [{'name': 'ESA', 'roles': ['producer'], 'url': 'https://esa.int/'}, {'name': 'AWS', 'roles': ['host'], 'url': 'https://aws.amazon.com/'}, {'name': 'Sinergise', 'roles': ['processor'], 'url': 'https://www.sinergise.com/'}], 'extent': {'spatial': {'bbox': [[-180.0, -56.0, 180.0, 83.0]]}, 'temporal': {'interval': [['2016-11-01T00:00:00Z', None]]}}, 'summaries': {'platform': ['sentinel-2a', 'sentinel-2b', 'sentinel-2c'], 'instrument': ['msi'], 'constellation': ['sentinel-2'], 'gsd': [10], 'eo:cloud_cover': {'minimum': 0, 'maximum': 100}, 'eo:bands': [{'

In [11]:
s2_coll

{'stac_version': '1.0.0',
 'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
  'https://stac-extensions.github.io/eo/v1.0.0/schema.json'],
 'type': 'Collection',
 'id': 'sentinel-2-l2a',
 'title': 'Sentinel 2 L2A',
 'description': 'Sentinel 2 imagery processed to level 2A',
 'sci:citation': 'Modified Copernicus Sentinel data [Year]/Sentinel Hub',
 'license': 'proprietary',
 'providers': [{'name': 'ESA',
   'roles': ['producer'],
   'url': 'https://esa.int/'},
  {'name': 'AWS', 'roles': ['host'], 'url': 'https://aws.amazon.com/'},
  {'name': 'Sinergise',
   'roles': ['processor'],
   'url': 'https://www.sinergise.com/'}],
 'extent': {'spatial': {'bbox': [[-180.0, -56.0, 180.0, 83.0]]},
  'temporal': {'interval': [['2016-11-01T00:00:00Z', None]]}},
 'summaries': {'platform': ['sentinel-2a', 'sentinel-2b', 'sentinel-2c'],
  'instrument': ['msi'],
  'constellation': ['sentinel-2'],
  'gsd': [10],
  'eo:cloud_cover': {'minimum': 0, 'maximum': 100},
  'eo

In [12]:
# lcfm-admin LCFM_100p_S2-tiles.fgb - LCFM_10p_S2-tiles.fgb
#

In [16]:
import geopandas as gpd

shapefiles_path = "/home/kalfasyan/Private/repos/lcfm-admin/shapefiles"
gdf100 = gpd.read_file(f"{shapefiles_path}/LCFM_100p_S2-tiles.fgb")
# eval(gdf100['bounds'].iloc[0]) - to create a bounding box
# crs - use epsg column
gdf100

,epsg,tile,bounds,geometry
0,32759,59FLB,"(300000.0, 4090240.0, 409800.0, 4200040.0)","POLYGON ((168.06583 -52.31348, 169.67594 -52.3..."
1,32758,58FEJ,"(499980.0, 4290220.0, 609780.0, 4400020.0)","POLYGON ((164.99972 -50.55175, 166.54931 -50.5..."
2,32757,57FWV,"(499980.0, 3890200.0, 609780.0, 4000000.0)","POLYGON ((158.99969 -54.1481, 160.68034 -54.13..."
3,32757,57FVV,"(399960.0, 3890200.0, 509760.0, 4000000.0)","POLYGON ((157.46869 -54.13837, 159.14943 -54.1..."
4,32757,57FVU,"(399960.0, 3790240.0, 509760.0, 3900040.0)","POLYGON ((157.43462 -55.03638, 159.15275 -55.0..."
...,...,...,...,...
18361,32709,09JUN,"(300000.0, 7190200.0, 409800.0, 7300000.0)","POLYGON ((-130.97225 -24.40058, -129.88965 -24..."
18362,32707,07JBK,"(199980.0, 6890200.0, 309780.0, 7000000.0)","POLYGON ((-144.02551 -27.08988, -142.9189 -27...."
18363,32706,06JYQ,"(699960.0, 6890200.0, 809760.0, 7000000.0)","POLYGON ((-144.9829 -27.10799, -143.87638 -27...."
18364,32701,01GEM,"(499980.0, 5090200.0, 609780.0, 5200000.0)","POLYGON ((-177.00025 -43.35286, -175.64555 -43..."


In [49]:
# Let's process the first tile
print(f"First tile: {gdf100.iloc[0].tile}")
tile_info = gdf100.iloc[0]
tile_bounds = tile_info['bounds']
tile_epsg = tile_info['epsg']
# Let's get the CRS from the EPSG code
from pyproj import CRS as pCRS
crs = pCRS.from_epsg(tile_epsg)
crs

First tile: 59FLB


<Projected CRS: EPSG:32759>
Name: WGS 84 / UTM zone 59S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 168°E and 174°E, southern hemisphere between 80°S and equator, onshore and offshore. New Zealand.
- bounds: (168.0, -80.0, 174.0, 0.0)
Coordinate Operation:
- name: UTM zone 59S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [79]:
from sentinelhub import CRS

In [ ]:
# CRS.UTM_59S


CRS('32759')

In [96]:
bounds = tuple(gdf100.bounds.iloc[0].to_list())

In [97]:
# Example search
bbox = BBox(bounds, crs=CRS('32759'))
data_collection = DataCollection.SENTINEL2_L2A
start_date = "2020-01-01"
end_date = "2020-03-31"
# end_date = "2020-12-31"
time_interval = (start_date, end_date)

search_iterator = catalog.search(
    collection=data_collection,
    bbox=bbox,
    time=time_interval,
    filter="eo:cloud_cover < 90",
    fields={
        "include": ["id", "properties.datetime", "properties.eo:cloud_cover"], # 
        "exclude": [],
    },
)

results = list(search_iterator)
print(f"Founds results: {len(results)}")
results

Founds results: 0


[]

In [15]:
catalog.get_feature(DataCollection.SENTINEL2_L2A, "S2A_MSIL2A_20210125T073201_N0214_R049_T39TWK_20210125T105105")

{'stac_version': '1.0.0',
 'stac_extensions': ['https://stac-extensions.github.io/eo/v1.0.0/schema.json',
  'https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
 'id': 'S2A_MSIL2A_20210125T073201_N0214_R049_T39TWK_20210125T105105',
 'type': 'Feature',
 'geometry': {'type': 'MultiPolygon',
  'crs': {'type': 'name',
   'properties': {'name': 'urn:ogc:def:crs:OGC::CRS84'}},
  'coordinates': [[[[50.99975828238186, 45.15382824779532],
     [50.99976235468337, 44.16537549736596],
     [52.37288170686682, 44.15712570690993],
     [52.39639823709577, 45.14529082814174],
     [50.99975828238186, 45.15382824779532]]]]},
 'bbox': [50.99975828238186,
  44.15712570690993,
  52.39639823709577,
  45.15382824779532],
 'properties': {'datetime': '2021-01-25T07:37:20Z',
  'platform': 'sentinel-2a',
  'instruments': ['msi'],
  'constellation': 'sentinel-2',
  'gsd': 10,
  'eo:cloud_cover': 0.59,
  'proj:epsg': 32639,
  'proj:bbox': [499980.0, 4890240.0, 609780.0, 5000040.0],
  'proj:geomet

In [16]:
catalog.get_collection(data_collection)

{'stac_version': '1.0.0',
 'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
  'https://stac-extensions.github.io/eo/v1.0.0/schema.json'],
 'type': 'Collection',
 'id': 'sentinel-2-l2a',
 'title': 'Sentinel 2 L2A',
 'description': 'Sentinel 2 imagery processed to level 2A',
 'sci:citation': 'Modified Copernicus Sentinel data [Year]/Sentinel Hub',
 'license': 'proprietary',
 'providers': [{'name': 'ESA',
   'roles': ['producer'],
   'url': 'https://esa.int/'},
  {'name': 'AWS', 'roles': ['host'], 'url': 'https://aws.amazon.com/'},
  {'name': 'Sinergise',
   'roles': ['processor'],
   'url': 'https://www.sinergise.com/'}],
 'extent': {'spatial': {'bbox': [[-180.0, -56.0, 180.0, 83.0]]},
  'temporal': {'interval': [['2016-11-01T00:00:00Z', None]]}},
 'summaries': {'platform': ['sentinel-2a', 'sentinel-2b', 'sentinel-2c'],
  'instrument': ['msi'],
  'constellation': ['sentinel-2'],
  'gsd': [10],
  'eo:cloud_cover': {'minimum': 0, 'maximum': 100},
  'eo